In [2]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [3]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
y_pred.std()

5.28140357655334

In [10]:
year = 2022
month = 2
output_file = f'yellow_pred_duration_{year:04d}-{month:02d}.parquet'

In [11]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [12]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['duration'] = y_pred

In [13]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)